In [ ]:
import numpy as np
import pandas as pd
import time
from multiprocessing import Pool
import statsmodels.formula.api as sm
import random
import itertools


import functions_compute_effect_sizes
import functions_loading_data
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
####### Options #############

# List sensitivity analyses
list_ancestries = ["All"]# "no_EUR" "EUR"
list_diagnosis = ["wo_ASD"] # "w_ASD" "All"
list_max_age = [12000]#, 75*12, 70*12, 65*12,  55*12
list_max_scores = [10000]#, 4, 6, 10, 20 ,40, 80

# Other options
cpu_count=62 # Number of CPUs to use
score_names = ["oe_lof_upper"]

other_covariates = ' + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10' # Other covariates to include in the model
pheno_score = 'ZScore_IQ_adj_test2_age_sex_with_online_UKBB_Final' # Name of the column containing the phenotype score (continuous variable)

####### Folder path containing all gene lists in tsv format
genesets_file = '<Path of the file containing the list of all genes in the coding genome>'

# Ouptut pathway
output = "<Path to store the results>"

In [ ]:
# Load the genesets file with two columns: the first one is the name of the geneset and the second one is the list of genes separated by ";"
genesets = pd.read_csv("{}".format(genesets_file), sep='\t')

In [ ]:
# Loading the dataset into 3 files, the first one containing the phenotypic information for each individual, the second one containing the list of CNV information for each individual and the third one containing the gene scores and information for each unique CNV.
# Only genes fully covered by the CNV are considered.
individual_info, cnv_indd, gene_score_info_overlap_split = functions_loading_data.prepare_Megadataset_dataframes(score_names)
# Load the list of genes associated to the DDD genes to be used as a covariate in the model
DDD_genes = functions_loading_data.get_DDD_genes()

In [ ]:
# Load all protein-coding genes with the a LOEUF score
genes_annotation = pd.read_csv('<Path to the dataframe containing the whole coding genome and the scores associated>', sep='\t', usecols=['gene_id', 'gene_type'] + score_names)
genes_annotation = genes_annotation[(~genes_annotation.oe_lof_upper.isna()) & (genes_annotation.gene_type=='protein_coding')]
print(genes_annotation.shape)

In [ ]:
# Create list of score names based on the given list of categories
cat_list = ['1_gene_list', '2_gene_list', '3_gene_list', '1_outside', '2_outside', 'ddd']
list_scores  = []
for cat in cat_list:
    list_scores.append("LOEUF_cat_{}".format(cat))
    
# Create dictonary containing list of unique genes carried by each individual for each category as key
list_uniques = functions_compute_effect_sizes.create_dict_uniques_genes_by_loeuf_cat(gene_score_info_overlap_split, cnv_indd)

In [ ]:
def compute_unique_gene_list(list_uniques, gene_list, carriers, cnv_type):
    list_uniques_type = {}
    for (key, value) in list_uniques.items():
        if key.startswith(cnv_type):
            list_uniques_type[key] = value

    uniques = {}

    loeuf_list = ['all']
    uniques['{}_all'.format(
        cnv_type)] = list_uniques_type['{}_all'.format(cnv_type)]


    c = carriers
    i = 0
    unique_genes = [[]]

    tmp_genes = uniques['{}_{}'.format(cnv_type, loeuf_list[i])][uniques['{}_{}'.format(
        cnv_type, loeuf_list[i])].individual.isin(c)].copy()
    all_genes = [genes.split(',') for genes in tmp_genes.gene_id]
    all_g = [g for sublist in all_genes for g in sublist]
    unique_genes.append([gl_g for gl_g in list(
        set(all_g)) if gl_g in gene_list.gene_id.tolist()])
    return unique_genes

In [ ]:
def runModel(
        data: pd.DataFrame,
        model_name: str,
        cnv_type: str,
        pheno_score: str,
        list_scores: list,
        gene_list: list,
        list_uniques: list,
        other_covariates: str) -> pd.DataFrame:
    
    if model_name == "1_3":
        tmp_scores = ['{}_{}'.format(cnv_type, val) for val in list_scores if not (val.endswith("gene_list"))]
        data.eval('{0}_LOEUF_gene_list = {0}_LOEUF_cat_1_gene_list + {0}_LOEUF_cat_2_gene_list + {0}_LOEUF_cat_3_gene_list'.format(cnv_type),
            inplace=True)
        formula = "{} ~ {}_LOEUF_gene_list + {}".format(pheno_score, cnv_type, ' + '.join(tmp_scores)) + other_covariates
        carriers = data[data['{0}_LOEUF_gene_list'.format(cnv_type)] > 0].individual.unique().tolist()
        unique_genes = compute_unique_gene_list(
            list_uniques, gene_list, carriers, cnv_type)

    
    if model_name == "3_3":
        tmp_scores = ['{}_{}'.format(cnv_type, val) for val in list_scores]
        formula = "{} ~ {}".format(pheno_score,
                                   ' + '.join(tmp_scores)) + other_covariates


    # Run the linear regression following formula and based on data. Capture error and print exception
    try:
        reg = sm.gls(formula, data=data).fit()
    except Exception as e:
        print("Error: ", e)
        return pd.DataFrame()

    return pd.DataFrame({'Estimate': reg.params[1:2],
                        'pvalue': reg.pvalues[1:2],
                        'TYPE': cnv_type,
                        'n_carriers': len(carriers),
						'n_unique_genes': len(unique_genes[1])
                         }
                        )

In [ ]:
# From the list of genes create a dictionary with the gene list as key and the list of genes as value
# The genesets are randomly generated from the list of all genes in the coding genome and their size varies from 10 to 6000 genes and for each size, 100 random gene sets are generated.
dict_r_genes = {}
all_genes = genesets.values[0][1].split(';')
print(len(all_genes))
for i in range(10, 6000,1):
    for j in range(100):
        dict_r_genes['{}_{}'.format(i,j)] = ';'.join(random.sample(all_genes, i))

df_r_genes = pd.DataFrame(dict_r_genes.items())
# Save the random gene sets into a tsv file
df_r_genes.to_csv('<Path to file where you want to store the gene-sets>', sep='\t', index=False)
print(df_r_genes)

In [ ]:
# Associate the list of genes fully deleted or duplicated for each individual
genes_by_individual = functions_compute_effect_sizes.aggregate_ind_cnv_genes(cnv_indd, gene_score_info_overlap_split.loc[:,['CHR', 'START', 'STOP', 'TYPE', 'gene_id']], individual_info)

In [ ]:
def compute_sum_score_one_gene_list(genes_annotation, genes_by_individual, gene_set):    
    print(gene_set[0])
    start_time = time.time()

    list_name = gene_set[0]
    gene_string = gene_set[1]
    genes = gene_string.split(';')
    gene_list = pd.DataFrame({'gene_id':genes})
    genes_annotation_gene_sets = functions_compute_effect_sizes.compute_loeuf_cat_specific_genes(genes_annotation, "gene_id", gene_list.iloc[:,0].tolist(), DDD_genes.gene_id.tolist())
    merged = pd.merge(genes_by_individual, genes_annotation_gene_sets, on='gene_id', how='left')
    counted_cnvs = {}
    clean_list_scores = list_scores+['loeuf_inv_full']
    for cnv_type in ['DEL', 'DUP']:
        merged_cnv = merged[merged.TYPE == cnv_type]
        counts = merged_cnv.groupby('individual', as_index=False)[clean_list_scores].sum()
        counts = counts.rename(columns={c: cnv_type+'_'+c for c in counts.columns if c in clean_list_scores})
        counted_cnvs[cnv_type] = counts

    combinations = list(itertools.product(['DEL', 'DUP'], clean_list_scores))
    joined_list = ['_'.join(combination) for combination in combinations]

    merged_counts = pd.merge(counted_cnvs['DEL'], counted_cnvs['DUP'], on='individual', how='outer')
    clean_FINAL_SUM_ALL = pd.merge(individual_info, merged_counts,  on='individual', how='left')
    clean_FINAL_SUM_ALL[joined_list] = clean_FINAL_SUM_ALL[joined_list].replace({np.nan:0.0})

    clean_FINAL_SUM_ALL = clean_FINAL_SUM_ALL[~clean_FINAL_SUM_ALL.Cohort.isin(["SSC", "SPARK", "MSSNG"])].copy()
    model_summary = pd.DataFrame()
    for cnv_type in ["DEL", "DUP"]:
        tmp_summary = runModel(clean_FINAL_SUM_ALL, "1_3", cnv_type, pheno_score, list_scores, gene_list,list_uniques, other_covariates)
        model_summary = pd.concat([model_summary, tmp_summary])

    model_summary["gene_list_name"] = gene_set[0]
    print("-- %s seconds --" % (time.time() - start_time), flush=True)
    return (model_summary)

In [ ]:
from functools import partial
partial_func = partial(compute_sum_score_one_gene_list, genes_annotation, genes_by_individual)

pool = Pool(int(cpu_count))
start_all = time.time()

data_outputs = pool.map(partial_func ,dict_r_genes.items())

pool.close()
pool.join()
print("-- %s seconds all--" % (time.time() - start_all))
df_reconstructed = pd.concat(data_outputs)

df_reconstructed.to_csv(output, sep="\t", index=False)